# Titanic - Machine Learning from Disaster

<img src="https://storage.googleapis.com/kaggle-competitions/kaggle/3136/logos/header.png" width=900>

- Vamos utilizar os [dados disponíveis no Kaggle](https://www.kaggle.com/competitions/titanic)
    - É um dataset de **competição**
    - O resultado é avaliado através da **acurácia**:
        - _"Sua pontuação é a porcentagem de passageiros que você prevê corretamente. Isso é conhecido como acurácia."_

### Importando novamente as bases e fazendo o tratamento dos dados
- Vamos apenas replicar o que fizemos no **[primeiro arquivo dessa análise](https://github.com/lucaslealx/Titanic/blob/main/Parte1.ipynb)** (para visualizar a parte 1 completa, visite esse link)

In [ ]:
# Importando o pandas
import pandas as pd

In [ ]:
# Visualizando a base de treino
treino = pd.read_csv('train.csv')
treino.head(3)

In [ ]:
# Visualizando a base de teste
teste = pd.read_csv('test.csv')
teste.head(3)

- **Fazendo o mesmo tratamento inicial que fizemos na aula anterior**

In [ ]:
# Eliminando as colunas com elevada cardinalidade
treino = treino.drop(['Name','Ticket','Cabin'],axis=1)
teste = teste.drop(['Name','Ticket','Cabin'],axis=1)

In [ ]:
# Usando a média para substituir valores nulos na coluna de idade
treino.loc[treino.Age.isnull(),'Age'] = treino.Age.mean()
teste.loc[teste.Age.isnull(),'Age'] = teste.Age.mean()

In [ ]:
# Tratando a coluna Embarked da base de treino usando a moda 
treino.loc[treino.Embarked.isnull(),'Embarked'] = treino.Embarked.mode()[0]

In [ ]:
# E também a coluna Fare da base de teste usando a média
teste.loc[teste.Fare.isnull(),'Fare'] = teste.Fare.mean()

### Podemos agora entender as colunas de texto
- Vamos agora **adicionar um novo tratamento das colunas de texto**

In [ ]:
# Verificando as colunas de texto na base de treino

In [ ]:
# Verificando os valores na coluna Sex

In [ ]:
# e na coluna Embarked

- Para tratar a coluna Sex, podemos criar uma nova coluna chamada **"Male_Check"** que **vai receber 1 se o gênero for masculino e 0 se o gênero for feminino**

In [ ]:
# Usando uma lambda function para fazer esse tratamento

In [ ]:
# Verificando os valores

- O próximo passo é fazer o mesmo para a base de teste

In [ ]:
# Usando uma lambda function para fazer esse tratamento

In [ ]:
# Verificando os valores

- Agora, para tratar a coluna **Embarked** usando o **[OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)** que irá **criar uma nova coluna para cada um dos rótulos da coluna original**

In [ ]:
# Importando o OneHotEncoder

In [ ]:
# Criando o encoder

In [ ]:
# Fazendo o fit com os dados

In [ ]:
# Fazendo a transformação

In [ ]:
# Transformando esse resultado em um DataFrame

In [ ]:
# Podemos agora adicionar essa coluna na nossa base de teste

In [ ]:
# Verificando os valores

- Agora vamos **fazer o mesmo para a base de teste usando o encoder ohe que criamos acima**

In [ ]:
# Transformando esse resultado em um DataFrame

In [ ]:
# Adicionando o resultado na base de teste

In [ ]:
# Verificando também os valores

### Usando essa nova base no modelo

In [ ]:
# Visualizando a base

In [ ]:
# Podemos então apagar essas duas colunas que já tratamos

- Podemos selecionar os mesmos modelos que vimos anteriormente (consulte arquivo da **[parte 1](https://github.com/lucaslealx/Titanic/blob/main/Parte1.ipynb)**)
    - **Árvore de classificação**
        - https://scikit-learn.org/stable/modules/tree.html#classification
    - **Classificação dos vizinhos mais próximos**
        - https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier
    - **Regressão Logística**
        - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression
- Antes de usar os algoritmos, precisamos separar a base de treino em **treino e validação**
    - Vamos fazer isso utilizando o **train_test_split**
        - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
# Importando o train_test_split
from sklearn.model_selection import train_test_split

In [ ]:
# Separando a base de treino em X e y
X = treino.drop(['PassengerId','Survived'],axis=1)
y = treino.Survived

In [ ]:
# Separando em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

- Para a **árvore de classificação**

In [ ]:
# Fazendo a importação
from sklearn import tree

In [ ]:
# Criando o classificador
clf_ac = tree.DecisionTreeClassifier(random_state=42)

In [ ]:
# Fazendo o fit com os dados
clf_ac = clf_ac.fit(X_train,y_train)

In [ ]:
# Fazendo a previsão
y_pred_ac = clf_ac.predict(X_val)

- Para o **KNeighborsClassifier**

In [ ]:
# Importando
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# Criando o classificador
clf_knn = KNeighborsClassifier(n_neighbors=3)

In [ ]:
# Fazendo o fit com os dados
clf_knn = clf_knn.fit(X_train,y_train)

In [ ]:
# Fazendo a previsão
y_pred_knn = clf_knn.predict(X_val)

- E para a **Regressão Logística**

In [ ]:
# Importando
from sklearn.linear_model import LogisticRegression

In [ ]:
# Criando o classificador
clf_rl = LogisticRegression(random_state=42,max_iter=1000)

In [ ]:
# Fazendo o fit com os dados
clf_rl = clf_rl.fit(X_train,y_train)

In [ ]:
# Fazendo a previsão
y_pred_rl = clf_rl.predict(X_val)

- Vamos novamente **avaliar os modelos**
    - Acurácia (método de avaliação usado na competição):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
    - Matriz de confusão (ajuda a visualizar a distribuição dos erros):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

- Avaliando a **acurácia**

In [ ]:
# Importando
from sklearn.metrics import accuracy_score

In [ ]:
# Para a árvore
accuracy_score(y_val, y_pred_ac)

In [ ]:
# Para o knn
accuracy_score(y_val, y_pred_knn)

In [ ]:
# Para a regressão logística
accuracy_score(y_val, y_pred_rl)

- Avaliando a **matriz de confusão**

In [ ]:
# Importando
from sklearn.metrics import confusion_matrix

In [ ]:
# Para a árvore
confusion_matrix(y_val, y_pred_ac)

In [ ]:
# Para o knn
confusion_matrix(y_val, y_pred_knn)

In [ ]:
# Para a regressão logística
confusion_matrix(y_val, y_pred_rl)

### Fazendo a previsão para os dados de teste
- Vamos usar o modelo com melhor precisão para fazer o predict na base de teste

In [ ]:
# Visualizando o X_train

In [ ]:
# Visualizando a base de teste

In [ ]:
# Para a base de teste ser igual a base de treino, precisamos eliminar a coluna de id

In [ ]:
# Utilizando a regressão logística na base de teste

In [ ]:
# Criando uma nova coluna com a previsão na base de teste

In [ ]:
# Selecionando apenas a coluna de Id e Survived para fazer o envio

In [ ]:
# Exportando para um csv